In [7]:
import math

sentence = ["i", "love", "deep", "learning"]
vocab = list(set(sentence))


word_to_index = {word: i for i, word in enumerate(vocab)}
index_to_word = {i: word for i, word in enumerate(vocab)}

vocab_size = len(vocab)
hidden_size = 2
learning_rate = 0.1


Wxh = [[0.1, 0.2, 0.1, 0.0], [0.0, 0.1, 0.2, 0.1]]
Whh = [[0.1, 0.2], [0.1, 0.1]]
Why = [[0.1, 0.1], [0.2, 0.1], [0.1, 0.1], [0.2, 0.2]]
bh = [0.0, 0.0]
by = [0.0, 0.0, 0.0, 0.0]

def one_hot(index, size):
    vec = [0] * size
    vec[index] = 1
    return vec

def tanh(x):
    return (2 / (1 + math.exp(-2 * x))) - 1

def dtanh(y):
    return 1 - y ** 2

def softmax(logits):
    exps = [math.exp(l) for l in logits]
    s = sum(exps)
    return [e / s for e in exps]

def train(inputs, target_index):
    x_s = []
    h_s = { -1: [0.0] * hidden_size }


    for t in range(len(inputs)):
        x = one_hot(inputs[t], vocab_size)
        x_s.append(x)

        h = []
        for i in range(hidden_size):
            z = sum(Wxh[i][j] * x[j] for j in range(vocab_size)) + \
                sum(Whh[i][j] * h_s[t-1][j] for j in range(hidden_size)) + bh[i]
            h.append(tanh(z))
        h_s[t] = h


    y = []
    for i in range(vocab_size):
        z = sum(Why[i][j] * h_s[len(inputs)-1][j] for j in range(hidden_size)) + by[i]
        y.append(z)
    probs = softmax(y)


    loss = -math.log(probs[target_index])


    dWhy = [[0 for _ in range(hidden_size)] for _ in range(vocab_size)]
    dby = [0] * vocab_size
    dy = probs[:]
    dy[target_index] -= 1


    for i in range(vocab_size):
        for j in range(hidden_size):
            dWhy[i][j] = dy[i] * h_s[len(inputs)-1][j]
        dby[i] = dy[i]


    dh = [sum(dy[o] * Why[o][h] for o in range(vocab_size)) for h in range(hidden_size)]
    dWxh = [[0 for _ in range(vocab_size)] for _ in range(hidden_size)]
    dWhh = [[0 for _ in range(hidden_size)] for _ in range(hidden_size)]
    dbh = [0 for _ in range(hidden_size)]

    for t in reversed(range(len(inputs))):
        dh_raw = [dh[i] * dtanh(h_s[t][i]) for i in range(hidden_size)]
        for i in range(hidden_size):
            for j in range(vocab_size):
                dWxh[i][j] += dh_raw[i] * x_s[t][j]
            for j in range(hidden_size):
                dWhh[i][j] += dh_raw[i] * h_s[t-1][j]
            dbh[i] += dh_raw[i]


    def clip(g, v=5):
        return max(-v, min(g, v))


    for i in range(hidden_size):
        for j in range(vocab_size):
            Wxh[i][j] -= learning_rate * clip(dWxh[i][j])
        for j in range(hidden_size):
            Whh[i][j] -= learning_rate * clip(dWhh[i][j])
        bh[i] -= learning_rate * clip(dbh[i])

    for i in range(vocab_size):
        for j in range(hidden_size):
            Why[i][j] -= learning_rate * clip(dWhy[i][j])
        by[i] -= learning_rate * clip(dby[i])

    return loss

inputs = [word_to_index["i"], word_to_index["love"], word_to_index["deep"]]
target = word_to_index["learning"]

for epoch in range(200):
    loss = train(inputs, target)
    if epoch % 20 == 0:
        print("Epoch", epoch, "Loss:", round(loss, 4))

def predict(input_words):
    input_indices = [word_to_index[word] for word in input_words]

    h_prev = [0.0 for _ in range(hidden_size)]
    for t in range(len(input_indices)):
        x = one_hot(input_indices[t], vocab_size)
        h = []
        for i in range(hidden_size):
            z = sum(Wxh[i][j] * x[j] for j in range(vocab_size)) + \
                sum(Whh[i][j] * h_prev[j] for j in range(hidden_size)) + bh[i]
            h.append(tanh(z))
        h_prev = h

    y = []
    for i in range(vocab_size):
        z = sum(Why[i][j] * h_prev[j] for j in range(hidden_size)) + by[i]
        y.append(z)
    probs = softmax(y)
    predicted_index = probs.index(max(probs))
    return index_to_word[predicted_index]

test_input = ["i", "love", "deep"]
print("\nPredicted word:", predict(test_input))

Epoch 0 Loss: 1.3943
Epoch 20 Loss: 0.4946
Epoch 40 Loss: 0.21
Epoch 60 Loss: 0.1088
Epoch 80 Loss: 0.0692
Epoch 100 Loss: 0.0492
Epoch 120 Loss: 0.0375
Epoch 140 Loss: 0.03
Epoch 160 Loss: 0.0248
Epoch 180 Loss: 0.021

Predicted word: learning
